# Advanced RAG Pipeline

[Reference RAG Link](https://docs.llamaindex.ai/en/latest/examples/retrievers/auto_merging_retriever.html#)

In [1]:
#import utils
import configparser
from pprint import pprint

# Read the INI file
config = configparser.RawConfigParser()
config.read("../config.ini")

# Access values from the sections
openai_api_section = config["OpenAI"]

OPENAI_API_KEY =openai_api_section["openai_key"] 

import os
import openai
# openai.api_key = OPENAI_API_KEY#utils.get_openai_api_key()
# openai.api_key
import utils
openai.api_key = utils.get_openai_api_key()
import jupyter_black
jupyter_black.load()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [2]:
#!curl https://www.gutenberg.org/cache/epub/47676/pg47676.txt > ../data/Amours.txt
#!ls ./../data
!head -100 ../data/Amours.txt

The Project Gutenberg eBook of The Amores; or, Amours
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Amores; or, Amours


Author: Ovid

Translator: Henry T. Riley

Release date: December 16, 2014 [eBook #47676]

Language: English

Credits: Produced by David Widger from page images generously
        provided by the Internet Archive


*** START OF THE PROJECT GUTENBERG EBOOK THE AMORES; OR, AMOURS ***



Produced by David Widger from page images generously
provided by the Internet Archive







THE AMORES;

or, AMOURS


By Ovid


Literally Translated into English Prose, with Copio

In [4]:
from llama_index import SimpleDirectoryReader

f1 = "./../data/RemediaAmoris.txt"
f2 = "./../data/METAMORPHOSES.txt"
f3 = "./../data/Heroides.txt"
f4 = "./../data/Amours.txt"
f5 = "./../data/Fasti.txt"
documents = SimpleDirectoryReader(input_files=[f1, f2, f3, f4, f5]).load_data()

In [5]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

5 

<class 'llama_index.schema.Document'>
Doc ID: b7e98ad7-02fe-41a4-9821-287e55abe1fc
Text: ﻿The Project Gutenberg eBook of Remedia Amoris; or, The Remedy
of Love      This ebook is for the use of anyone anywhere in the
United States and most other parts of the world at no cost and with
almost no restrictions whatsoever. You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this ebook...


## Basic RAG pipeline

In [6]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [7]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document], service_context=service_context)

In [8]:
query_engine = index.as_query_engine()

In [ ]:
# Uncomment: Sanity-check response
# response = query_engine.query(
#     "Who is Aglauros?"
# )
# pprint(str(response))

## Evaluation setup using TruLens

In [32]:
eval_questions = [
    "Who is Aglauros?",
    "What does Thebes want?",
    "What is the Bœotian city?",
    "What is the context of the quote: 'Resist beginnings; the remedy comes too late when'?",
    "Who said: They bear punishment with equanimity who have earned it.",
    "Please explain what this means: They bear punishment with equanimity who have earned it."
    "Find a quote similar to the following: 'Resist beginnings; the remedy comes too late when'?",
    "Who said: 'We're slow to believe what wounds us.'",
    "Who is The Queen of Erebus?",
    "Where is the Who is The Queen of Erebus mentioned?",
]
# You can try your own question:
new_question = "Who said: No one ought to be pronounced happy before his death, and his last obsequies.?"
eval_questions.append(new_question)

In [10]:
from trulens_eval import Tru

tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [11]:
from trulens_eval import Feedback, TruLlama, OpenAI

# OpenAI(api_key=OPENAI_API_KEY)
# help(OpenAI)

In [12]:
from utils import get_prebuilt_trulens_recorder


tru_recorder = get_prebuilt_trulens_recorder(query_engine, app_id="Direct Query Engine")

In [13]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [15]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_0e3c6b387c6b968103837d4d1e87ef62,"""Who is Aglauros?""","""Aglauros is a character mentioned in the cont...",-,"{""record_id"": ""record_hash_0e3c6b387c6b9681038...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-02T08:08:57.745957"", ""...",2023-12-02T08:08:59.649741,1.0,1.0,0.0,"[{'args': {'prompt': 'Who is Aglauros?', 'resp...",[{'args': {'source': '[Footnote 79: _Plains of...,"[{'args': {'prompt': 'Who is Aglauros?', 'resp...",1,2027,0.003046
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_af04a180cc54921419b6bc69febe5f84,"""What does Thebes want?""","""Thebes wants to regain its honor and expel it...",-,"{""record_id"": ""record_hash_af04a180cc54921419b...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-02T08:08:59.753115"", ""...",2023-12-02T08:09:01.514673,1.0,1.0,0.0,"[{'args': {'prompt': 'What does Thebes want?',...","[{'args': {'source': 'Do be mindful, I entreat...","[{'args': {'prompt': 'What does Thebes want?',...",1,2058,0.003095
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_2ed9819abf9a1484a8307421ce340353,"""What is the B\u0153otian city?""","""The B\u0153otian city mentioned in the contex...",-,"{""record_id"": ""record_hash_2ed9819abf9a1484a83...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-02T08:09:01.587091"", ""...",2023-12-02T08:09:03.150352,1.0,1.0,0.0,[{'args': {'prompt': 'What is the Bœotian city...,[{'args': {'source': '[Footnote 12: _A mournfu...,[{'args': {'prompt': 'What is the Bœotian city...,1,1749,0.002632
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_1099f02b8c3aa37134dee738474d6815,"""What is the context of the quote: 'Resist beg...","""The context of the quote \""Resist beginnings;...",-,"{""record_id"": ""record_hash_1099f02b8c3aa37134d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-02T08:09:03.224422"", ""...",2023-12-02T08:09:10.260443,1.0,1.0,NaN,[{'args': {'prompt': 'What is the context of t...,[{'args': {'source': 'For time supplies streng...,NaN,7,2167,0.003292
4,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_fd80c6e0541c45a67c6f80daa93a3dcf,"""Who said: They bear punishment with equanimit...","""Dionysius said: \""They bear punishment with e...",-,"{""record_id"": ""record_hash_fd80c6e0541c45a67c6...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-02T08:09:10.334914"", ""...",2023-12-02T08:09:11.849733,1.0,0.0,0.0,[{'args': {'prompt': 'Who said: They bear puni...,"[{'args': {'source': 'Krebs, has, I think, she...",[{'args': {'prompt': 'Who said: They bear puni...,1,2068,0.003111


In [16]:
for idx, row in records.iterrows():
    print(row["input"])
    print()
    print(row["output"])
    print()
    print()

"Who is Aglauros?"

"Aglauros is a character mentioned in the context."


"What does Thebes want?"

"Thebes wants to regain its honor and expel its feeble foe."


"What is the B\u0153otian city?"

"The B\u0153otian city mentioned in the context is Aganippe."


"What is the context of the quote: 'Resist beginnings; the remedy comes too late when'?"

"The context of the quote \"Resist beginnings; the remedy comes too late when\" is about the importance of taking action and resisting the initial stages of a passion or desire. It emphasizes that if one hesitates for too long, the problem or malady will become stronger and more difficult to cure. The quote encourages individuals to be proactive and not postpone addressing issues, as delaying action will only make the situation worse."


"Who said: They bear punishment with equanimity who have earned it."

"Dionysius said: \"They bear punishment with equanimity who have earned it.\""


"Please explain what this means: They bear punishment wi

In [17]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.4.62:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG Pipeline
### 1. Sentence Window Retrieval

In [18]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.5)

In [19]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
)

In [20]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [21]:
window_response = sentence_window_engine.query("Who is Aglauros?")
print(str(window_response))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Aglauros is a character who is approached by Mercury in order to gain access to her sister, Herse. She refuses to help him unless he promises to give her a large sum of money.


In [35]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine, app_id="Sentence Window Query Engine"
)

In [36]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))
        print("\n\n")

Who is Aglauros?
Aglauros is a character who refuses to assist Mercury in his pursuit of Herse, unless he promises to give her a large sum of money.



What does Thebes want?
Thebes wants the Theban ladies to go.



What is the Bœotian city?
The Bœotian city is not mentioned in the given context information.



What is the context of the quote: 'Resist beginnings; the remedy comes too late when'?
The context of the quote "Resist beginnings; the remedy comes too late when" cannot be determined based on the provided information.



Who said: They bear punishment with equanimity who have earned it.
The speaker who said, "They bear punishment with equanimity who have earned it," is not mentioned in the given context information.



Please explain what this means: They bear punishment with equanimity who have earned it.Find a quote similar to the following: 'Resist beginnings; the remedy comes too late when'?
They bear punishment with equanimity who have earned it means that those who have 

In [39]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.75,0.0,0.0,2.1,0.000735


In [40]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.4.62:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-Merging Retrieval

In [27]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="merging_index"
)

In [28]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [29]:
auto_merging_response = automerging_query_engine.query("Who is The Queen of Erebus?")
print(str(auto_merging_response))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The Queen of Erebus is Persephone.


In [30]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(
    automerging_query_engine, app_id="Automerging Query Engine"
)

In [31]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

> Merging 1 nodes into parent node.
> Parent node id: ccae6d21-c328-408d-ae97-92083798eebe.
> Parent node text: Hyginus also speaks of her as being
    the daughter of Oceanus. From the name ‘Nysa,’ Bacchus re...



Who is Aglauros?
Aglauros is a character mentioned in the text.


What does Thebes want?
Thebes wants an unarmed boy to take over the city.
> Merging 1 nodes into parent node.
> Parent node id: 8a72d278-21cd-4835-b1e9-d992f7d63744.
> Parent node text: 228. This and Amphrysus were
    rivers of Thessaly.]

    [Footnote 27: _Shores of Bœbe._--Ver. ...



What is the Bœotian city?
The Bœotian city is a city that was instructed to be built by someone under the guidance of a certain individual.


What is the context of the quote: 'Resist beginnings; the remedy comes too late when'?
The context of the quote "Resist beginnings; the remedy comes too late when" cannot be determined based on the provided context information.


Who said: They bear punishment with equanimity who have earned it.
Pentheus said: They bear punishment with equanimity who have earned it.


Please explain what this means: They bear punishment with equanimity who have earned it.Find a quote similar to the following: 'Resist beginnings; the remedy comes too late when'?
Those who have earned punishment bear it with equanimity. It means that when someone has done something to deserve punishment, they accept it calmly and without complaint. They understand that they are responsible for their actions and are willing to face the consequences. 

A quote similar to "Resist beginnings; the remedy comes too late when" could be "Prevention is better than cure."


Who said: 'We're slow to believe what wounds us.'
The speaker who said, "We're slow to believe what wounds us," is not mentioned in the given context information.


Who said: No one ought to be pronounced happy before his death, and his last obsequies.?
Solon said: No one ought to be pronounced happy before his death, and his last obsequies.


In [33]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.6125,0.2,0.5,2.5,0.000549


In [34]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.4.62:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>